<div style="direction:rtl;line-height:300%;">
<font face="XB Zar" size=5>
<div align=center>
<font face="B Titr" size=5>
<p></p><p></p>
بسمه تعالی
<p></p>
</font>
<p></p>
<font>
<br>
درس بازیابی پیشرفته اطلاعات
<br>
مدرس: دکتر بیگی
</font>
<p></p>
<br>
<font>
<b>تمرین دوم</b>
</font>
<br>
<br>
موعد تحویل: ... آبان <br>
<br>
<br>
<br>
<font>
دانشگاه صنعتی شریف
<br>
دانشکده مهندسی کامپیوتر
<br>
<br>
</font>
</div>
</font>
</div>

<div dir='rtl'>
<h1> 
مقدمه
</h1>
<p>
در این تمرین قصد داریم به مباحث زیر بپردازیم:
    <li>مدل‌های برداری</li>
    <li>امتیازدهی و ارزیابی سیستم بازیابی</li>
    <li>مدل‌های احتمالاتی</li>

دیتاست مورد استفاده در این تمرین را می‌توانید در کنار این فایل مشاهده کنید. همچنین لطفا پس از اتمام تمرین یک بار از اول تا آخر نوت‌بوک را اجرا کنید تا مطمئن باشید تمام سل‌ها به درستی کار می‌کنند. 

کتابخانه‌های مورد نظرتان را هم می‌توانید در اولین سل نوت‌بوک فراخوانی کنید. 
</p>
</div>

In [2]:
import math
import pandas as pd
import csv
import json
import numpy as np
from collections import defaultdict
from typing import List, Union
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.metrics import edit_distance
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/mhdi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mhdi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mhdi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

<div dir='rtl'>
<h1>1.
آماده‌سازی دیتاست
</h1>
<p>
با استفاده از تمرین قبل و عملیات‌هایی که در آنجا برای پیش‌پردازش متون پیاده‌سازی کردید، دیتاست داده شده را لود کنید. قرار است در ادامه با این دیتاست بخش‌های بعدی تمرین را پیاده‌سازی کنید. همچنین در صورتی که اجرای سل‌های بعدی برایتان طول کشید، می‌توانید آن را کوچک کنید. مثلا از ۷۰درصد دیتای آن استفاده کنید. 
</p>
</div>

In [3]:
class Preprocessor:

    def __init__(self, stopwords):
        self.stopwords = stopwords
        self.ps = PorterStemmer()
        self.lem = WordNetLemmatizer()


    def preprocess(self, text, is_query=False):
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        tokens = self.word_tokenize(text)
        tokens = self.normalize(tokens)
        if not is_query:
            tokens = self.remove_stopwords(tokens)
        return tokens

    def normalize(self, tokens):
        for i, token in enumerate(tokens):
            token = token.lower()
            token = self.ps.stem(token)
            tokens[i] = self.lem.lemmatize(token)
        return tokens
    
    def remove_links(self, text):
        text = re.sub('http[s]?://\S+', '', text)        
        return text

    def remove_punctuations(self, text):
        regular_expression = r'[^\w\s]'
        return re.sub(regular_expression, ' ', text.lower())

    def word_tokenize(self, text):
        tokenized_text = nltk.tokenize.word_tokenize(text)
        return tokenized_text

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

preprocessor = Preprocessor(stopwords=stopwords.words('english'))

In [4]:
## 1 points

data = {}
all_tokens = []
index_to_paperId = {}
paperId_to_index = {}
index = 1

with open('data/phase-2/data-v1.csv', 'r', encoding="utf-8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for it, row in enumerate(csv_reader):
        if it == 0:
            continue
            
        paperId = row[0]
        title = row[1]
        abstract = row[2]

        title_tokens = preprocessor.preprocess(title)
        abstract_tokens = preprocessor.preprocess(abstract)

        all_tokens.append(title_tokens)
        all_tokens.append(abstract_tokens)

        index_to_paperId[index] = paperId
        paperId_to_index[paperId] = index
        data[index] = (title_tokens, abstract_tokens)
        index += 1


<div dir='rtl'>
<h2>1-1.
نمایه‌سازی
</h2>
<p>
در این بخش باید برای سامانه یک
Positional Index
بسازید. 
<br>
با توجه به مواردی که در بخش بعد می‌آید و نیاز به جست‌وجو‌ی مجزا و با امتیازدهی متفاوت بر روی بخش‌های مختلف سند مثل عنوان یا چکیده آن، در این قسمت باید نمایه‌ی مناسب برای امکان جست‌وجو‌ در بخش‌های مختلف را پیاده‌سازی کنید.
با استفاده از نمایه‌ی ساخته‌شده باید بتوان شماره‌ی تمامی اسنادی که یک کلمه در آن آمده است و همچنین همه‌ی جایگاه‌های این کلمه در هر بخش از هر سند را پیدا کرد.

برای این بخش می‌توانید از نمایه‌ای که در تمرین اول زدید استفاده کنید. 
</p>
</div>

In [5]:
def construct_positional_indexes(data: defaultdict):

    """
    Get processed data and insert words in that into a trie and construct postional_index and posting lists after wards.

    Parameters
    ----------
    corpus: str
        processed data 
    
    Return
    ----------
    docs: 
        list of docs with specificied id, title, abstract.
    """
    index = {}
    for paper_id, (title, abstract) in data.items():
        for i, word in enumerate(title):
            if word not in index:
                index[word] = {'title': {}, 'abstract': {}}
            if word not in index[word]['title']:
                index[word]['title'][paper_id] = []
            index[word]['title'][paper_id].append(i)

        for i, word in enumerate(abstract):
            if word not in index:
                index[word] = {'title': {}, 'abstract': {}}
            if word not in index[word]['abstract']:
                index[word]['abstract'][paper_id] = []
            index[word]['abstract'][paper_id].append(i)

    return index

positional_index = construct_positional_indexes(data)

In [6]:
print(positional_index)

{'analysi': {'title': {1: [0], 7: [1], 8: [7], 16: [2], 34: [0], 42: [14], 71: [2], 73: [8], 76: [13]}, 'abstract': {13: [49], 15: [71], 25: [37], 31: [178], 34: [119], 53: [76], 54: [127], 65: [107], 72: [150], 73: [85], 74: [575], 76: [205], 93: [58], 99: [90], 101: [49]}}, 'ground': {'title': {1: [1]}, 'abstract': {}}, 'surfac': {'title': {1: [2], 9: [3]}, 'abstract': {11: [6], 45: [114], 105: [135]}}, 'ultrason': {'title': {1: [3]}, 'abstract': {}}, 'face': {'title': {1: [4], 18: [0], 29: [1]}, 'abstract': {18: [154], 29: [98], 39: [49], 56: [3]}}, 'grind': {'title': {1: [5]}, 'abstract': {}}, 'silicon': {'title': {1: [6]}, 'abstract': {}}, 'carbid': {'title': {1: [7]}, 'abstract': {}}, 'sic': {'title': {1: [8]}, 'abstract': {}}, 'ceram': {'title': {1: [9]}, 'abstract': {70: [41]}}, 'minor': {'title': {1: [10]}, 'abstract': {}}, 'vibrat': {'title': {1: [11]}, 'abstract': {}}, 'amplitud': {'title': {1: [12]}, 'abstract': {}}, 'measur': {'title': {2: [0], 44: [7], 48: [0], 83: [2]}, 

<div dir='rtl'>
<h1>2.
مدل‌های برداری و احتمالاتی
</h1>
<p>
در این بخش قصد داریم تا با استفاده از مدل‌های برداری و احتمالاتی، دو سیستم بازیابی اطلاعات طراحی کنیم. در نهایت قرار است سیستم‌های طراحی شده در این بخش را مورد ارزیابی قرار دهیم.

در مدل‌های برداری ما به ازای هر داک که در اختیار داریم و کوئری ورودی یک بردار در فضا در نظر می‌گیریم. در ادامه برای بدست آوردن میزان ارتباط داک‌ها و کوئری، از معیارهایی مانند ضرب داخلی بردارها استفاده می‌کنیم. در این بخش ابتدا بردارهای مربوط به هرکدام را با استفاده از مقادیر 
tf, idf
می‌سازیم و سپس به سراغ محاسبه امتیاز داک‌ها می‌رویم تا در نهایت بتوانیم با استفاده از این امتیازها داک‌هایی مرتبط‌تر را خروجی دهیم و یک سیستم بازیابی کامل را طراحی کنیم. 

در مدل‌های احتمالاتی اساس کار محاسبه 
$P(R | d, q)$
است. در این روش می‌خواهیم به نوعی داک‌ها را به صورت مدل‌های احتمالاتی ببینیم. در درس مشاهده کردید که در انتها مدل‌های احتمالاتی هم رفتاری مشابه با مدل‌های برداری دارند. در ادامه یک سیستم مبتنی بر این مدل‌ها و با روش 
Okapi25
طراحی می‌کنید. 
</p>
</div>

<div dir='rtl'>
<h2>2-1.
ساخت ماتریس TF-IDF
</h2>
<p>
در این مرحله دو تابع
<code dir="ltr">get_tf(token, doc_id)</code>
و
<code dir="ltr">get_idf(token)</code>
را پیاده‌سازی کنید که تابع اول مقدار
tf
توکن ورودی را در شناسه‌ی موردنظر
و تابع دوم مقدار
idf
توکن ورودی را برروی نمایه‌ی کنونی حساب می‌کند و آن را به عنوان خروجی برمی‌گرداند.
</p>
</div>

$$tf_{t, d} = \text{Numbers of } t \text{ in the title of document } d + \text{Numbers of } t \text{ in the abstract of document } d$$
$$idf_t = \log\Biggl(\frac{\text{Number of documents}}{\text{Number of documents that contains } t + 1}\Biggr)$$

In [12]:
def get_tf_doc(token: str, positional_index: dict, doc_id: int) -> int:
    """
    Retrieves the term frequency (TF) of a given token within a specific document.

    Parameters
    ----------
    token : str
        The token for which to retrieve the term frequency.
    positional_index : dict
        A positional index that maps tokens to their positions in documents.
    doc_id : str
        The unique identifier of the document in which to calculate the TF.

    Returns
    -------
    int
        The term frequency (TF) of the given token within the specified document.
    """
    if token not in positional_index:
        return 0
    tf = 0

    if doc_id in positional_index[token]['abstract']:
        tf += len(positional_index[token]['abstract'][doc_id])
    if doc_id in positional_index[token]['title']:
        tf += len(positional_index[token]['title'][doc_id])
    return tf

def get_tf_query(query) -> dict:
    query_tf = {}
    for token in query:
        if token not in query_tf:
            query_tf[token] = 0
        query_tf[token] += 1
    
    return query_tf

def get_tf_for_all_docs(token: str, positional_index: dict):
    tf = 0
    for doc in paperId_to_index.keys():
        tf += get_tf_doc(token, positional_index, doc)
    
    return tf


get_tf_doc('analysi', positional_index, 1)

1

In [13]:
def get_idf(token: str) -> float:
    """
    Retrieves the inverse document frequency (IDF) of a given token.

    Parameters
    ----------
    token : str
        The token to retrieve the IDF for.

    Returns
    -------
    float
        The IDF of the given token.
    """

    df = len(np.union1d(
        np.array(positional_index[token]['abstract'].keys()),
        np.array(positional_index[token]['title'].keys())
    ))
    
    return np.log10(len(index_to_paperId) / df)

get_idf('deep')

1.7708520116421442

<div dir='rtl'>
<p>
در مرحله‌ی نهایی تابع
<code dir="ltr">generate_tfidf_list(corpus, positional_index)</code>
را پیاده‌سازی کنید که با ورودی گرفتن
corpus
و
positional_index
مقادیر
tf-idf
را برای هر
token
به دست می‌آورد و به شکل یک ماتریس به عنوان خروجی برمی‌گرداند.
</p>
</div>

In [14]:
def generate_tfidf_list(positional_index: dict) -> list[dict[str, float]]:
    """
    Generates a list of dictionaries representing documents with associated TF-IDF scores.

    Parameters
    ----------
    corpus : dict
        The corpus containing the processed data.
    positional_index : dict
        The positional index containing the term frequencies and document frequencies.

    Returns
    -------
    list[dict[str, float]]
        The list of dictionaries representing documents with associated TF-IDF scores.
    """
    tf_idf = {}
    for token in positional_index.keys():
        if token not in tf_idf:
            tf_idf[token] = {}
        for doc in positional_index[token]['abstract']:
            idf = get_idf(token)
            tf = np.log10(get_tf_doc(token, positional_index, doc)) + 1
            tf_idf[token][doc] = tf * idf
        for doc in positional_index[token]['title']:
            idf = get_idf(token)
            tf = np.log10(get_tf_doc(token, positional_index, doc)) + 1
            if doc in tf_idf[token]:
                tf_idf[token][doc] += tf * idf
            else:
                tf_idf[token][doc] = tf * idf
    
    return tf_idf
        

tf_idf = generate_tfidf_list(positional_index)

In [15]:
print(tf_idf)

{'analysi': {13: 1.7708520116421442, 15: 1.7708520116421442, 25: 1.7708520116421442, 31: 1.7708520116421442, 34: 4.607863170056663, 53: 1.7708520116421442, 54: 1.7708520116421442, 65: 1.7708520116421442, 72: 1.7708520116421442, 73: 4.607863170056663, 74: 1.7708520116421442, 76: 4.607863170056663, 93: 1.7708520116421442, 99: 1.7708520116421442, 101: 1.7708520116421442, 1: 1.7708520116421442, 7: 1.7708520116421442, 8: 1.7708520116421442, 16: 1.7708520116421442, 42: 1.7708520116421442, 71: 1.7708520116421442}, 'ground': {1: 1.7708520116421442}, 'surfac': {11: 1.7708520116421442, 45: 1.7708520116421442, 105: 1.7708520116421442, 1: 1.7708520116421442, 9: 1.7708520116421442}, 'ultrason': {1: 1.7708520116421442}, 'face': {18: 4.607863170056663, 29: 4.607863170056663, 39: 1.7708520116421442, 56: 1.7708520116421442, 1: 1.7708520116421442}, 'grind': {1: 1.7708520116421442}, 'silicon': {1: 1.7708520116421442}, 'carbid': {1: 1.7708520116421442}, 'sic': {1: 1.7708520116421442}, 'ceram': {70: 1.7708

<div dir='rtl'>
<h2>2-2.
امتیازدهی به سندها
</h2>
<p>
در این بخش می‌خواهیم تا با سه شیوه‌ی متفاوت، داکیومنت‌ها را امتیازدهی کرده و kتا با بالاترین امتیاز را برگردانیم.
روش‌ها به شکل زیر است:

- **ltc.lnc:** در این روش از cosine similarity استفاده می‌کنیم و داده‌ها را بر اساس آن مرتّب می‌کنیم. برای متوجّه شدن نحوه‌ی عملکرد این سیستم، می‌توانید به اسلایدهای درس مراجعه کنید. همچنین دقت کنید که لازم است این امتیازدهی را هم به روش 
term-at-a-time
و هم به روش 
doc-at-a-time
محاسبه کنید.

- **Okapi25:** برای پیاده‌سازی این روش از تساوی زیر استفاده کنید:
</p>
</div>
<div>
<p>

$$ RSV_d = \sum_{t \in q} idf(t)\times\frac{(k_1 + 1)tf(t, d)}{k_1((1 - b) + b\frac{dl(d)}{avg(dl)}) + tf(t, d)} $$
</p>
</div>
<div dir='rtl'>
<p>
برای محاسبه این دو معیار، از توابعی که در بخش قبل پیاده‌سازی کردید استفاده کنید.

در تابع search که مربوط به جست و جوی پرسمان کلی است، به عنوان ورودی پارامتر پرسمان (query)، روش محاسبه امتیاز (method)، تعداد اسنادی که باید برگردانده شود (max-result-count) را ورودی می گیرید. ورودی وزن 
(weight)
نشان می‌دهد که امتیاز نهایی چه وزنی از امتیاز عنوان و چکیده دارد. به زبان دیگر:
</p>
</div>
<div>
<p>

$$ final\_score = weight \times title\_score + (1 - weight) \times abstract\_score $$
</p>
</div>
<div dir='rtl'>
<p>

**توجّه کنید** که تابع نوشته شده، صرفاً یک prototype است و تا وقتی که نیازمندی‌های گفته شده را پیاده کنید، تغییر تابع مشکلی ندارد.
</p>
</div>

In [20]:
def ltc_lnc(query_tokens, positional_index, tf_idf, how='abstract'):
    query_tf = get_tf_query(query_tokens)
    query_tf_idf = {}
    for token in query_tokens:
        query_tf_idf[token] = (np.log10(query_tf[token]) + 1) * get_idf(token)
    
    doc_scores = {}

    for token in query_tokens:
        docs = positional_index[token][how]
        for doc in docs:
            if doc not in doc_scores:
                doc_scores[doc] = 0
            doc_scores[doc] += query_tf_idf[token] * tf_idf[token][doc]

    return doc_scores


In [21]:
query = "Translation Model Based on Deep Learning"
query_tokens = preprocessor.preprocess(query)
print(ltc_lnc(query_tokens, positional_index, tf_idf))

{74: 9.407750541411087, 100: 6.271833694274058, 3: 11.295760611203619, 10: 3.135916847137029, 17: 9.407750541411087, 18: 16.31968752813318, 21: 9.407750541411087, 28: 11.295760611203619, 30: 22.591521222407238, 39: 19.45560437527021, 40: 17.567594305477677, 43: 3.135916847137029, 45: 3.135916847137029, 47: 6.271833694274058, 48: 19.45560437527021, 53: 3.135916847137029, 54: 3.135916847137029, 57: 3.135916847137029, 62: 9.407750541411087, 64: 6.271833694274058, 69: 6.271833694274058, 72: 9.407750541411087, 78: 9.407750541411087, 82: 14.431677458340648, 84: 3.135916847137029, 94: 6.271833694274058, 98: 16.31968752813318, 109: 14.431677458340648, 116: 6.271833694274058, 2: 3.135916847137029, 4: 8.15984376406659, 13: 3.135916847137029, 16: 8.15984376406659, 25: 11.295760611203619, 27: 19.45560437527021, 29: 19.45560437527021, 32: 3.135916847137029, 41: 3.135916847137029, 46: 3.135916847137029, 56: 3.135916847137029, 65: 3.135916847137029, 73: 9.407750541411087, 89: 3.135916847137029, 97: 9

In [ ]:
def okapi_bm25(query_tokens, positional_index, k1=1.2, b=0.75, how='abstract'):
    doc_scores = {}

    for token in query_tokens:
        docs = positional_index[token][how]
        for doc in docs:
            if doc not in doc_scores:
                doc_scores[doc] = 0
            doc_scores[doc] += get_idf(token) * ((k1 + 1) * get_tf_doc(token, positional_index, doc)) / (get_tf_doc(token, positional_index, doc) + k1 * (1 - b + b * len(positional_index[token][how][doc]) / 100))

    return doc_scores

In [ ]:
def merge_doc_scores(doc_scores1: dict, doc_scores2: dict, weight: float) -> dict:
    merged_doc_scores = {}
    for doc in doc_scores1.keys():
        merged_doc_scores[doc] = weight * doc_scores1[doc]

    for doc in doc_scores2.keys():
        if doc in merged_doc_scores:
            merged_doc_scores[doc] += (1 - weight) * doc_scores2[doc]
        else:
            merged_doc_scores[doc] = (1 - weight) * doc_scores2[doc]

    return merged_doc_scores

def search(query: str, max_result_count: int, method: str = 'ltc-lnc', weight: float = 0.5):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_resu/lt_count = -1, then you have to return all docs

    method: 'ltc-lnc' or 'okapi25'

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """
    query_token = preprocessor.preprocess(query)
    if method == 'ltc-lnc':
        doc_scores_abstract = ltc_lnc(query_token, positional_index, tf_idf, how='abstract')
        doc_scores_title = ltc_lnc(query_token, positional_index, tf_idf, how='title')
        doc_scores = merge_doc_scores(doc_scores_title, doc_scores_abstract, weight)
    
    elif method == 'okapi25':
        doc_scores_abstract = okapi_bm25(query_token, positional_index, tf_idf, how='abstract')
        doc_scores_title = okapi_bm25(query_token, positional_index, tf_idf, how='title')
        doc_scores = merge_doc_scores(doc_scores_title, doc_scores_abstract, weight)
    
    sorted_docs = [index_to_paperId[d] for d in sorted(doc_scores.items(), key=lambda x:x[1], reverse=True)] 
    if len(sorted_docs) < max_result_count:
        return sorted_docs
    return sorted_docs[:max_result_count]

<div dir='rtl'>
<h1>3.
مدل‌های زبانی
</h1>
<p>
در این بخش یک سیستم مبتنی بر مدل‌های زبانی پیاده‌سازی می‌کنید. مدل زبانی‌ای که باید استفاده کنید مدل 
unigram
است که می‌توان گفت ساده‌ترین مدل ممکن است. در این مدل ما برای هر داک یک مدل دظر نظر می‌گیریم و احتمال حضور کلمات در مدل یک داک 
نسبت به هم مستقل هستند. در پیاده‌سازی این بخش دستتان خیلی باز است. مدل‌هایی که تشکیل می‌دهید و نحوه 
smoothing 
و همچنین پارامترهای مدل همگی به خودتان برمی‌گردد. 
</p>
</div>

In [ ]:
def lm_search(query: str, max_result_count: int):
    """
    Finds relevant documents to query

    Parameters
    ---------------------------------------------------------------------------------------------------
    max_result_count: Return top 'max_result_count' docs which have the highest scores.
                        notice that if max_result_count = -1, then you have to return all docs

    Returns
    ----------------------------------------------------------------------------------------------------
    list
    Retreived documents with snippet
    """
    pass

<div dir='rtl'>
<h1>4.
ارزیابی عملکرد سامانه
</h1>
<p>
در این بخش معیارهای زیر را برای سیستم‌های طراحی شده پیاده‌سازی کنید. سپس در بخش آخر با فراخوانی توابع پیاده‌سازی شده و ورودی دادن مقادیر موجود در فایل
validation.json
به عنوان ورودی 
actual
توابع، می‌توانید معیارها را برای سیستم‌های بازیابی خودتان بدست بیاورید. 
</p>
</div>

In [ ]:
class Evaluation:
    def evaluate(self, require_scores, actual, predicted):
        """
        Prints require scores for actual and predicts array

        Parameters
        ----------
        require_scores : List[str]
            Scores required to calculated

        Returns
        -------
        """
        for method in require_scores:
            func = getattr(Evaluation, method)
            print(f'method:{func(actual, predicted)}')

    @staticmethod
    def precision(actual, predicted):
        """
        Calculates the precision of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The precision of the predicted results
        """
        precision = 0

        for i in range(len(predicted)):
            actual_set = actual[i]
            predicted_set = predicted[i]
            precision += len(set(actual_set) & set(predicted_set)) / len(predicted[i])
        
        return precision / len(actual)

    @staticmethod
    def recall(actual, predicted) -> float:
        """
        Calculates the recall of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The recall of the predicted results
        """
        recall = 0

        for i in range(len(actual)):
            actual_set = actual[i]
            predicted_set = predicted[i]

            recall += len(set(actual_set) & set(predicted_set)) / len(actual[i])
        
        return recall / len(actual)

    @staticmethod
    def f1(actual, predicted) -> float:
        """
        Calculates the F1 score of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The F1 score of the predicted results    
        """
        precision = Evaluation.precision(actual, predicted)
        recall = Evaluation.recall(actual, predicted)

        return (2 * precision * recall) / (precision + recall)

    @staticmethod
    def map(actual, predicted) -> float:
        """
        Calculates the Mean Average Precision of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The Mean Average Precision of the predicted results
        """
        map = 0.0

        for i in range(len(actual)):
            actual_set = actual[i]
            predicted_set = predicted[i]

            precision = 0.0
            for j in range(len(predicted_set)):
                if predicted_set[j] in actual_set:
                    precision += Evaluation.precision(actual_set, predicted_set[:j+1])
            
            map += precision / len(actual_set)

        return map

    @staticmethod
    def ndcg(actual, predicted) -> float:
        """
        Calculates the Normalized Discounted Cumulative Gain (NDCG) of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The NDCG of the predicted results
        """
        ndcg = 0.0

        for i in range(len(actual)):
            actual_set = actual[i]
            predicted_set = predicted[i]

            dcg = 0.0
            idcg = 0.0
            for j in range(len(predicted_set)):
                if predicted_set[j] in actual_set:
                    dcg += 1 / np.log2(j + 2)
                idcg += 1 / np.log2(j + 2)
            
            ndcg += dcg / idcg

        return ndcg / len(actual)
    
    @staticmethod
    def mrr(actual, predicted) -> float:
        """
        Calculates the Mean Reciprocal Rank of the predicted results

        Parameters
        ----------
        actual : List[List[str]]
            The actual results
        predicted : List[List[str]]
            The predicted results

        Returns
        -------
        float
            The MRR of the predicted results
        """
        MRR = 0.0

        for i in range(len(actual)):
            actual_set = actual[i]
            predicted_set = predicted[i]

            for j in range(len(predicted_set)):
                if predicted_set[j] in actual_set:
                    MRR += 1 / (j + 1)
                    break

        return MRR / len(actual)

In [ ]:
require_scores = ["precision", "recall", "f1", "map", "ndcg", "mrr"]

with open('./data/phase-2/validation.json', 'r') as f:
    validation = json.load(f)

queries = list(validation.keys())
actuals = [validation[query] for query in queries]

In [ ]:
predicts = [search(query, 10, method='ltc-lnc') for query in queries]

Evaluation().evaluate(require_scores, actuals, predicts)

In [ ]:
predicts = [search(query, 10, method='okapi25') for query in queries]

Evaluation().evaluate(require_scores, actuals, predicts)

In [ ]:
predicts = [lm_search(query, 10) for query in queries]

Evaluation().evaluate(require_scores, actuals, predicts)